**2025 Military AI CONtest**<br>
AI Autonomous Driving – Pre-Training Code Sample<br> 
NVIDIA Jetson Nano/Wingbot/JupyterLab<br>
Version : v1.0<br>
File : robot_line_following_test.ipynb<br>
**회색선 추종 테스트 코드**


**1. 라이브러리 및 초기화**<br>


In [ ]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import subprocess
import warnings
import os
from tiki.mini import TikiMini

# GStreamer 경고 메시지 필터링
# "Cannot query video position" 경고는 실시간 카메라 스트림에서 정상적인 메시지입니다.
# 비디오 파일이 아닌 실시간 스트림은 위치 정보가 없기 때문에 나타나는 경고입니다.
# 이 경고는 무시해도 되며, 카메라 기능에는 영향을 주지 않습니다.
warnings.filterwarnings('ignore', category=UserWarning)
# GStreamer 디버그 메시지 최소화 (필요시 '0'으로 설정하여 에러만 표시)
os.environ['GST_DEBUG'] = '0'  # 0=에러만, 1=경고, 2=정보

# 로봇 초기화
tiki = TikiMini()

# 모터 모드 설정 (PWM 모드)
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)

print("로봇 초기화 완료")

# 카메라 파이프라인 설정 (여러 옵션 시도)
pipeline_options = [
    # 옵션 1: 기본 파이프라인
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=30/1 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
    # 옵션 2: 낮은 프레임레이트
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=15/1 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
    # 옵션 3: 추가 옵션 없이
    (
        "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=320, height=240, format=NV12 ! "
        "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
    ),
]

cap = None
pipeline_used = None

# 카메라 초기화 시도
print("카메라 초기화 시도 중...")

# 먼저 다른 프로세스가 카메라를 사용 중인지 확인
try:
    result = subprocess.run(['fuser', '/dev/video0'], capture_output=True, text=True, timeout=1)
    if result.returncode == 0:
        print("[경고] 카메라가 다른 프로세스에서 사용 중일 수 있습니다.")
except:
    pass

# 각 파이프라인 옵션 시도
for i, pipeline in enumerate(pipeline_options, 1):
    print(f"파이프라인 옵션 {i} 시도 중...")
    cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
    
    # 카메라가 열릴 때까지 잠시 대기
    time.sleep(0.5)
    
    if cap.isOpened():
        # 테스트 프레임 읽기
        ret, test_frame = cap.read()
        if ret:
            print(f"✓ 카메라 연결 성공! (파이프라인 옵션 {i} 사용)")
            pipeline_used = i
            break
        else:
            print(f"파이프라인 옵션 {i}: 프레임 읽기 실패")
            cap.release()
            cap = None
    else:
        print(f"파이프라인 옵션 {i}: 카메라 열기 실패")
        if cap:
            cap.release()
            cap = None

if cap is None or not cap.isOpened():
    print("\n[에러] 카메라 초기화 실패")
    print("가능한 해결 방법:")
    print("1. 다른 Jupyter 노트북이나 프로세스가 카메라를 사용 중인지 확인")
    print("2. Jetson Nano를 재부팅")
    print("3. 카메라 하드웨어 연결 확인")
    print("4. 권한 확인: sudo usermod -a -G video $USER")
    raise RuntimeError("카메라를 열 수 없습니다. 위의 해결 방법을 확인하세요.")

# 비디오 위젯 설정 (반드시 display() 전에 설정)
video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='320px', height='240px'))

# 비디오 위젯 표시 (display는 초기화 후 한 번만 호출)
display(video_widget)

# 비디오 위젯이 제대로 표시되었는지 확인
print("비디오 위젯 표시 완료")
print(f"비디오 위젯 타입: {type(video_widget)}")
print("초기화 완료")


**2-0. 회색선 검출 코드 분석**<br>
<br>
**회색선 검출 알고리즘 단계별 분석:**<br>
1. **ROI 설정**: 프레임의 하단 40% 영역만 처리 (차선 인식에 적합)<br>
2. **그레이스케일 변환**: 컬러 이미지를 흑백으로 변환<br>
3. **가우시안 블러**: 노이즈 제거 (5x5 커널)<br>
4. **임계값 처리**: 회색 범위 검출 (100-200 밝기 범위)<br>
   - 밝은 회색: 100 이상<br>
   - 어두운 회색: 200 이하<br>
5. **마스크 결합**: 두 임계값 마스크를 AND 연산<br>
6. **픽셀 위치 추출**: 회색 픽셀의 X, Y 좌표 찾기<br>
7. **중심 계산**: 회색 픽셀들의 평균 X 좌표 계산<br>
<br>
**파라미터:**<br>
- `threshold_low = 100`: 회색 하한 (조명 환경에 따라 조정 가능)<br>
- `threshold_high = 200`: 회색 상한 (조명 환경에 따라 조정 가능)<br>
- `ROI`: 하단 40%, 좌우 10-75% (경로에 따라 조정 가능)<br>


**2-1. 회색선 검출 디버그 테스트 (선택사항)**<br>
<br>
**회색선 검출 함수를 테스트하고 로그를 확인하세요**<br>


In [ ]:
# 회색선 검출 함수 테스트 (디버그 모드)
# 카메라에서 프레임을 읽어 회색선 검출 과정을 단계별로 확인

if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다. 먼저 초기화 셀을 실행하세요.")
else:
    print("회색선 검출 테스트 시작...")
    print("=" * 60)
    
    # 몇 프레임 테스트
    for test_i in range(5):
        ret, test_frame = cap.read()
        if ret:
            test_frame = cv2.flip(test_frame, -1)
            print(f"\n[테스트 프레임 {test_i+1}]")
            print("-" * 60)
            
            # 디버그 모드로 검출
            result = detect_gray_line(test_frame, debug=True, show_images=False)
            
            if len(result) == 4:
                line_center_x, mask, roi, debug_info = result
            else:
                line_center_x, mask, roi = result
                debug_info = None
            
            if line_center_x is not None:
                print(f"✓ 라인 검출 성공: 중심 X = {line_center_x}")
            else:
                print("✗ 라인 검출 실패")
            
            if debug_info:
                print(f"\n[디버그 정보 요약]")
                print(f"  - 그레이스케일 평균: {debug_info.get('gray', {}).get('mean', 0):.1f}")
                print(f"  - 마스크 픽셀 비율: {debug_info.get('mask', {}).get('percentage', 0):.2f}%")
            
            time.sleep(0.5)
        else:
            print(f"프레임 {test_i+1} 읽기 실패")
    
    print("\n" + "=" * 60)
    print("테스트 완료")


**2. 회색선 검출 함수**<br>


In [ ]:
# 회색선 검출 디버그 설정
DETECT_GRAY_LINE_DEBUG = False  # True로 설정하면 상세 로그 출력
DETECT_GRAY_LINE_SHOW_IMAGES = False  # True로 설정하면 중간 이미지 표시

def detect_gray_line(frame, debug=False, show_images=False):
    """
    회색선을 검출하고 중심 좌표를 반환하는 함수
    
    Parameters:
    frame: 입력 프레임 (BGR)
    debug: 디버그 모드 (상세 로그 출력)
    show_images: 중간 처리 이미지 표시
    
    Returns:
    line_center_x: 라인 중심 X 좌표 (없으면 None)
    mask: 검출된 마스크
    roi: ROI 영역
    debug_info: 디버그 정보 딕셔너리 (debug=True일 때)
    """
    debug_info = {}
    
    # 1단계: ROI 설정
    h, w = frame.shape[:2]
    roi_y1 = int(h * 0.60)
    roi_y2 = h
    roi_x1 = int(w * 0.10)
    roi_x2 = int(w * 0.75)
    
    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]
    
    if debug:
        debug_info['roi'] = {
            'frame_size': (h, w),
            'roi_bounds': (roi_x1, roi_y1, roi_x2, roi_y2),
            'roi_size': (roi_h, roi_w)
        }
        print(f"[1단계] ROI 설정: 프레임 크기={h}x{w}, ROI={roi_x1},{roi_y1}~{roi_x2},{roi_y2}, ROI 크기={roi_h}x{roi_w}")
    
    # 2단계: 그레이스케일 변환
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_mean = np.mean(gray)
    gray_std = np.std(gray)
    
    if debug:
        debug_info['gray'] = {
            'mean': gray_mean,
            'std': gray_std,
            'min': int(np.min(gray)),
            'max': int(np.max(gray))
        }
        print(f"[2단계] 그레이스케일 변환: 평균={gray_mean:.1f}, 표준편차={gray_std:.1f}, 범위=[{int(np.min(gray))}, {int(np.max(gray))}]")
    
    if show_images:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 3))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
        plt.title('ROI')
        plt.axis('off')
    
    # 3단계: 가우시안 블러 (노이즈 제거)
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    if debug:
        blurred_mean = np.mean(gray_blurred)
        print(f"[3단계] 가우시안 블러: 평균={blurred_mean:.1f} (변화: {blurred_mean-gray_mean:.1f})")
    
    # 4단계: 회색선 검출 (임계값 범위: 100-200)
    threshold_low = 100  # 회색 하한
    threshold_high = 200  # 회색 상한
    
    _, mask_light = cv2.threshold(gray_blurred, threshold_low, 255, cv2.THRESH_BINARY)  # 밝은 회색
    _, mask_dark = cv2.threshold(gray_blurred, threshold_high, 255, cv2.THRESH_BINARY_INV)  # 어두운 회색
    
    if debug:
        light_count = np.sum(mask_light == 255)
        dark_count = np.sum(mask_dark == 255)
        debug_info['threshold'] = {
            'low': threshold_low,
            'high': threshold_high,
            'light_pixels': light_count,
            'dark_pixels': dark_count
        }
        print(f"[4단계] 임계값 처리: 범위=[{threshold_low}, {threshold_high}], "
              f"밝은 회색 픽셀={light_count}, 어두운 회색 픽셀={dark_count}")
    
    if show_images:
        plt.subplot(1, 3, 2)
        plt.imshow(gray_blurred, cmap='gray')
        plt.title('Gray + Blur')
        plt.axis('off')
    
    # 5단계: 두 마스크를 결합하여 회색 범위 검출
    mask = cv2.bitwise_and(mask_light, mask_dark)
    mask_count = np.sum(mask == 255)
    mask_percentage = (mask_count / (roi_h * roi_w)) * 100
    
    if debug:
        debug_info['mask'] = {
            'pixel_count': mask_count,
            'percentage': mask_percentage
        }
        print(f"[5단계] 마스크 결합: 회색 픽셀={mask_count}개 ({mask_percentage:.2f}%)")
    
    if show_images:
        plt.subplot(1, 3, 3)
        plt.imshow(mask, cmap='gray')
        plt.title('Final Mask')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    # 6단계: 회색 픽셀 위치 찾기
    ys, xs = np.where(mask == 255)
    
    if len(xs) > 0:
        # ROI 좌표계에서의 중심
        line_center_x_roi = int(np.mean(xs))
        line_center_y_roi = int(np.mean(ys))
        line_std_x = np.std(xs)
        
        # 전체 프레임 좌표계로 변환
        line_center_x = roi_x1 + line_center_x_roi
        line_center_y = roi_y1 + line_center_y_roi
        
        if debug:
            debug_info['detection'] = {
                'found': True,
                'center_roi': (line_center_x_roi, line_center_y_roi),
                'center_frame': (line_center_x, line_center_y),
                'std_x': line_std_x,
                'pixel_count': len(xs),
                'x_range': (int(np.min(xs)), int(np.max(xs))),
                'y_range': (int(np.min(ys)), int(np.max(ys)))
            }
            print(f"[6단계] 라인 검출 성공:")
            print(f"  - ROI 좌표계: 중심=({line_center_x_roi}, {line_center_y_roi}), X 표준편차={line_std_x:.1f}")
            print(f"  - 프레임 좌표계: 중심=({line_center_x}, {line_center_y})")
            print(f"  - 픽셀 수: {len(xs)}개")
            print(f"  - X 범위: [{int(np.min(xs))}, {int(np.max(xs))}], Y 범위: [{int(np.min(ys))}, {int(np.max(ys))}]")
        
        if debug or DETECT_GRAY_LINE_DEBUG:
            return line_center_x, mask, roi, debug_info
        else:
            return line_center_x, mask, roi
    else:
        if debug:
            debug_info['detection'] = {
                'found': False,
                'reason': 'No gray pixels detected'
            }
            print(f"[6단계] 라인 검출 실패: 회색 픽셀이 없습니다")
        
        if debug or DETECT_GRAY_LINE_DEBUG:
            return None, mask, roi, debug_info
        else:
            return None, mask, roi

print("회색선 검출 함수 초기화 완료")
print("디버그 모드: detect_gray_line(frame, debug=True, show_images=True)")


**3. PID 제어 클래스**<br>


In [ ]:
class PIDController:
    """PID 제어기 클래스"""
    
    def __init__(self, kp=0.5, ki=0.0, kd=0.1):
        """
        Parameters:
        kp: 비례 게인 (Proportional gain)
        ki: 적분 게인 (Integral gain)
        kd: 미분 게인 (Derivative gain)
        """
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0
        self.previous_error = 0
    
    def compute(self, error):
        """
        PID 제어 출력 계산
        
        Parameters:
        error: 오차 (라인 중심 - 프레임 중심)
        
        Returns:
        output: 제어 출력값
        """
        # 적분 항 (누적 오차)
        self.integral += error
        
        # 미분 항 (오차 변화율)
        derivative = error - self.previous_error
        
        # PID 출력 계산
        output = (self.kp * error) + (self.ki * self.integral) + (self.kd * derivative)
        
        # 적분 항 제한 (Windup 방지)
        if self.integral > 100:
            self.integral = 100
        elif self.integral < -100:
            self.integral = -100
        
        self.previous_error = error
        
        return output

# PID 제어기 생성 (튜닝 가능)
pid = PIDController(kp=0.8, ki=0.0, kd=0.2)

print("PID 제어기 초기화 완료")


**3-1. 카메라 테스트 (선택사항)**<br>
<br>
**카메라가 제대로 작동하는지 먼저 테스트해보세요**<br>


In [ ]:
# 카메라 테스트 (선택사항)
# 메인 루프 전에 카메라가 제대로 작동하는지 확인

def frame_to_bytes(frame):
    """프레임을 JPEG 바이트로 변환"""
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다.")
elif video_widget is None:
    print("[에러] 비디오 위젯이 초기화되지 않았습니다. 초기화 셀을 다시 실행하세요.")
else:
    print("카메라 테스트 중...")
    test_count = 0
    max_test = 10
    
    for i in range(max_test):
        ret, test_frame = cap.read()
        if ret:
            test_count += 1
            # 테스트 프레임을 비디오 위젯에 표시
            test_frame_flipped = cv2.flip(test_frame, -1)
            video_widget.value = frame_to_bytes(test_frame_flipped)
            print(f"✓ 프레임 {i+1}/{max_test} 읽기 성공 (크기: {test_frame.shape})")
            time.sleep(0.1)
        else:
            print(f"✗ 프레임 {i+1}/{max_test} 읽기 실패")
            time.sleep(0.1)
    
    if test_count > 0:
        print(f"\n✓ 카메라 정상 작동 확인 ({test_count}/{max_test} 프레임 성공)")
        print("메인 루프 실행 가능")
    else:
        print("\n✗ 카메라 프레임 읽기 실패")
        print("카메라 연결을 확인하세요.")


**4. 회색선 추종 메인 루프**<br>


In [ ]:
def frame_to_bytes(frame):
    """프레임을 JPEG 바이트로 변환"""
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

# 주행 파라미터
base_speed = 25  # 기본 속도 (0-127)
max_steering = 20  # 최대 조향값
frame_center_x = 160  # 프레임 중앙 X 좌표 (320/2)

print("회색선 추종 시작 (Ctrl+C로 종료)")
print(f"기본 속도: {base_speed}, 최대 조향: {max_steering}")

# 카메라 및 비디오 위젯 확인
if cap is None or not cap.isOpened():
    print("[에러] 카메라가 초기화되지 않았습니다. 먼저 초기화 셀을 실행하세요.")
elif video_widget is None:
    print("[에러] 비디오 위젯이 초기화되지 않았습니다. 초기화 셀을 다시 실행하세요.")
else:
    print("비디오 위젯 확인 완료")
    try:
        frame_count = 0
        error_count = 0
        max_errors = 10  # 최대 연속 에러 횟수
        
        while True:
            # 카메라 연결 상태 확인
            if not cap.isOpened():
                print("[경고] 카메라 연결이 끊어졌습니다.")
                error_count += 1
                if error_count > max_errors:
                    print("[에러] 카메라 연결 실패. 종료합니다.")
                    break
                time.sleep(0.5)
                continue
            
            ret, frame = cap.read()
            if not ret:
                error_count += 1
                print(f"[경고] 프레임 읽기 실패 ({error_count}/{max_errors})")
                if error_count > max_errors:
                    print("[에러] 연속 프레임 읽기 실패. 종료합니다.")
                    break
                time.sleep(0.1)
                continue
            
            # 프레임을 읽었으면 에러 카운터 리셋
            error_count = 0
            frame_count += 1
            
            # 프레임 상하반전 (카메라 설치 방향에 따라 조정)
            frame = cv2.flip(frame, -1)
            
            # 회색선 검출 (디버그 모드 옵션)
            # 매 N번째 프레임마다 디버그 로그 출력
            debug_this_frame = (frame_count % 30 == 0)  # 30프레임마다 디버그
            show_images_this_frame = False  # 이미지는 표시하지 않음 (성능 위해)
            
            result = detect_gray_line(frame, debug=debug_this_frame, show_images=show_images_this_frame)
            
            if len(result) == 4:
                line_center_x, mask, roi, debug_info = result
            else:
                line_center_x, mask, roi = result
                debug_info = None
            
            if line_center_x is not None:
                # 오차 계산 (라인 중심 - 프레임 중심)
                error = line_center_x - frame_center_x
                
                # PID 제어 출력 계산
                steering = pid.compute(error)
                
                # 조향값 제한
                steering = np.clip(steering, -max_steering, max_steering)
                
                # 좌우 모터 속도 계산
                left_speed = base_speed + steering
                right_speed = base_speed - steering
                
                # 속도 제한 (0-127)
                left_speed = int(np.clip(left_speed, 0, 127))
                right_speed = int(np.clip(right_speed, 0, 127))
                
                # 모터 제어
                tiki.set_motor_power(tiki.MOTOR_LEFT, left_speed)
                tiki.set_motor_power(tiki.MOTOR_RIGHT, right_speed)
                
                # 디버깅 정보 표시
                cv2.circle(frame, (line_center_x, int(frame.shape[0] * 0.8)), 5, (0, 255, 0), -1)  # 라인 중심
                cv2.circle(frame, (frame_center_x, int(frame.shape[0] * 0.8)), 5, (0, 0, 255), -1)  # 프레임 중심
                cv2.line(frame, (line_center_x, int(frame.shape[0] * 0.8)), 
                         (frame_center_x, int(frame.shape[0] * 0.8)), (255, 0, 0), 2)  # 오차선
                
                # ROI 영역 표시
                h, w = frame.shape[:2]
                roi_y1 = int(h * 0.60)
                roi_y2 = h
                roi_x1 = int(w * 0.10)
                roi_x2 = int(w * 0.75)
                cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), (255, 255, 0), 2)
                
                # 정보 텍스트 표시
                cv2.putText(frame, f"Error: {error:.1f}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Steering: {steering:.1f}", (10, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"L:{left_speed} R:{right_speed}", (10, 70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
            else:
                # 라인을 찾지 못한 경우 정지
                tiki.stop()
                cv2.putText(frame, "Line not detected - STOP", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # 비디오 위젯 업데이트
            try:
                frame_bytes = frame_to_bytes(frame)
                if frame_bytes is not None and len(frame_bytes) > 0:
                    video_widget.value = frame_bytes
                else:
                    print("[경고] 프레임 바이트 변환 실패")
            except Exception as e:
                print(f"[경고] 비디오 위젯 업데이트 실패: {e}")
            
            # 디버깅: 처음 5프레임 또는 디버그 프레임일 때 정보 출력
            if frame_count <= 5 or debug_this_frame:
                if line_center_x is not None:
                    print(f"[프레임 {frame_count}] 라인 검출 성공: 중심={line_center_x}, 오차={line_center_x-frame_center_x:.1f}")
                    if debug_info and 'mask' in debug_info:
                        print(f"  - 마스크 픽셀: {debug_info['mask']['pixel_count']}개 ({debug_info['mask']['percentage']:.2f}%)")
                else:
                    print(f"[프레임 {frame_count}] 라인 검출 실패")
                    if debug_info and 'gray' in debug_info:
                        print(f"  - 그레이스케일 평균: {debug_info['gray']['mean']:.1f}")
            
            time.sleep(0.02)  # ~50 FPS

    except KeyboardInterrupt:
        print("\n회색선 추종 종료")

    except Exception as e:
        print(f"\n[에러] 예외 발생: {e}")
        print("예외 타입:", type(e).__name__)

    finally:
        # 정지 및 리소스 해제
        print("\n리소스 해제 중...")
        tiki.stop()
        if cap is not None:
            cap.release()
        print("리소스 해제 완료")
        if 'frame_count' in locals():
            print(f"총 처리된 프레임: {frame_count}")


**5. 파라미터 튜닝 가이드**<br>
<br>
**조정 가능한 파라미터:**<br>
- `base_speed`: 기본 주행 속도 (15-30 권장)<br>
- `max_steering`: 최대 조향값 (15-25 권장)<br>
- `PID 파라미터`: kp, ki, kd 값 조정<br>
  - kp 증가 → 빠른 반응, 불안정<br>
  - ki 증가 → 누적 오차 보정, 오버슈트 가능<br>
  - kd 증가 → 진동 억제, 지연 가능<br>
- `임계값 범위`: 회색선 검출 범위 (100-200)<br>
- `ROI 영역`: 검출 영역 조정<br>
<br>
**디버깅 정보:**<br>
- 녹색 원: 라인 중심 위치<br>
- 빨간 원: 프레임 중심 위치<br>
- 파란 선: 오차 (라인 중심과 프레임 중심 사이)<br>
- 노란 사각형: ROI 영역<br>
- 텍스트: Error, Steering, 좌우 모터 속도<br>
